# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import f_oneway
from scipy.stats import skew, iqr, zscore
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df=pd.read_csv('bank-additional-full.csv',delimiter=';')

In [3]:
df.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,24,technician,single,professional.course,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [5]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [6]:
df.shape

(41188, 21)

In [7]:
print(df['y'].value_counts())
print(df['y'].nunique())
print()
print('-'*100)
df["y"] = df['y'].replace({'yes':1,'no':0})

print(df['y'].value_counts())
print(df['y'].nunique())

y
no     36548
yes     4640
Name: count, dtype: int64
2

----------------------------------------------------------------------------------------------------
y
0    36548
1     4640
Name: count, dtype: int64
2


In [8]:
numeric_data = df.select_dtypes(include=[np.number])

categorical_data = df.select_dtypes(exclude=[np.number])


In [9]:
categorical_data.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome'],
      dtype='object')

In [10]:
print(f'Data contains {df.shape[0]} samples and {df.shape[1]} variables')

#ID_col=
TARGET_COL='y'

#features = [c for c in data.columns if c not in [ID_COL, TARGET_COL]]
features = [c for c in df.columns if c not in [TARGET_COL]]
print(f'\nThe dataset contains {len(features)} input features')

Data contains 41188 samples and 21 variables

The dataset contains 20 input features


# Statistical Analysis on Continuous Variables

In [11]:
numeric_data.head(5)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0


In [12]:
numeric_data.tail(5)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,1
41187,74,239,3,999,1,-1.1,94.767,-50.8,1.028,4963.6,0


In [13]:
numeric_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   duration        41188 non-null  int64  
 2   campaign        41188 non-null  int64  
 3   pdays           41188 non-null  int64  
 4   previous        41188 non-null  int64  
 5   emp.var.rate    41188 non-null  float64
 6   cons.price.idx  41188 non-null  float64
 7   cons.conf.idx   41188 non-null  float64
 8   euribor3m       41188 non-null  float64
 9   nr.employed     41188 non-null  float64
 10  y               41188 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 3.5 MB


# Convergence Rate (Percentage)

In [14]:
d1=numeric_data[numeric_data['y']==1]


In [15]:
# Assuming d1 is your DataFrame and numeric_data contains 'age' column
Conversion_Age_Yes = (d1['age'].value_counts() / numeric_data['age'].value_counts()) * 100

# Convert to DataFrame
conversion_df_age = pd.DataFrame({
    'Conversion Percentage in Yes': Conversion_Age_Yes.values
}, index=Conversion_Age_Yes.index)

# Rename the index column to 'age'
conversion_df_age.index.name = 'Age'

# Display the DataFrame
conversion_df_age

,Conversion Percentage in Yes
Age,
17,40.000000
18,42.857143
19,47.619048
20,35.384615
21,28.431373
...,...
91,NaN
92,75.000000
94,NaN


# Proportion of Term Deposit Subscribers by Occupation

In [16]:
data1= df.copy()

data1['y'] = data1['y'].replace(['no','yes'],[0,1]) #needed to do below mathematical operations
data1.head()

#no of clients who subscribed to the term deposit grouped by occupation
total_subscribers = data1.groupby('job').y.sum().values

# Proportion of clients who subscribed to the term deposit grouped by occupation
proportion_subscribed = (round(data1.groupby('job').y.sum()/data1.groupby('job').y.count(),3)*100).values

# Total amount of clients per occupation
total_people = data1.groupby('job').y.count().values

#Form a dataframe and print
jobs = sorted(data1.job.unique()) #list of all jobs
jobs_with_subscribers = pd.DataFrame({'Job': jobs, 'Total Subscribers':total_subscribers,'Total People in Job': total_people,'Proportion of Subscribers': proportion_subscribed})
jobs_with_subscribers.sort_values(by='Proportion of Subscribers', ascending=False)

,Job,Total Subscribers,Total People in Job,Proportion of Subscribers
8,student,275,875,31.4
5,retired,434,1720,25.2
10,unemployed,144,1014,14.2
0,admin.,1352,10422,13.0
4,management,328,2924,11.2
11,unknown,37,330,11.2
9,technician,730,6743,10.8
6,self-employed,149,1421,10.5
3,housemaid,106,1060,10.0
2,entrepreneur,124,1456,8.5


# Marital Status and Campaign Success Rate Comparison

In [17]:
married_subscribers = data1[(data1.marital=='married') ].y.sum()
single_subscribers = data1[(data1.marital=='single') | (data1.marital=='divorced')].y.sum() #single or divorced

married_subscribers_prop = married_subscribers/len(data1[data1.marital=='married'])
single_subscribers_prop = single_subscribers/len(data1[(data1.marital=='single') | (data1.marital=='divorced')])

print('No of Married clients who subscribe: {}'.format(married_subscribers))
print('No of Single (and divorced) clients who subscribe : {}'.format(single_subscribers))

print('Married clients campaign success rate: {0:.0f}%'.format(married_subscribers_prop*100))
print('Single clients campaign success rate: {0:.0f}%'.format(single_subscribers_prop*100))

No of Married clients who subscribe: 2532
No of Single (and divorced) clients who subscribe : 2096
Married clients campaign success rate: 10%
Single clients campaign success rate: 13%


# Age Group and Subscription Proportion Analysis

In [18]:
n1=len(numeric_data[(numeric_data['age'] > 60) & (numeric_data['y'] == 1)]) #number of old subscribers
n2=len(numeric_data[(numeric_data['age'] > 60) & (numeric_data['y'] == 0)]) #number of old non-subscribers
n3=len(numeric_data[(numeric_data['age'] <= 60) & (numeric_data['y'] == 1)]) #number of young subscribers
n4=len(numeric_data[(numeric_data['age'] <= 60) & (numeric_data['y'] == 0)]) #number of young non-subscribers

print(f'Proportion of young subscribers is: {n3/(n3+n4)} and the proportion of old subscribers is {n1/(n1+n2)}')

Proportion of young subscribers is: 0.10492080043696311 and the proportion of old subscribers is 0.45494505494505494


# Conversion Percentage Based on __ Days Before Called Analysis

In [19]:
# Assuming d1 is your DataFrame and numeric_data contains 'pdays' column
Conversion_Pdays_Yes = (d1['pdays'].value_counts() / numeric_data['pdays'].value_counts()) * 100

# Convert to DataFrame
conversion_df = pd.DataFrame({
    'Conversion Percentage in Yes': Conversion_Pdays_Yes.values
}, index=Conversion_Pdays_Yes.index)

# Rename the index column to 'days'
conversion_df.index.name = 'Days Before Called'

# Display the DataFrame
conversion_df

,Conversion Percentage in Yes
Days Before Called,
0,66.666667
1,30.769231
2,60.655738
3,67.881549
4,53.389831
5,63.043478
6,70.145631
7,66.666667
8,66.666667


# Conversion Percentage Based on Previous Calls Analysis

In [20]:
# Assuming d1 is your DataFrame and numeric_data contains 'previous' column
Conversion_Previous_Yes = (d1['previous'].value_counts() / numeric_data['previous'].value_counts()) * 100

# Convert to DataFrame
conversion_previous_df = pd.DataFrame({
    'Conversion Percentage in Yes': Conversion_Previous_Yes.values
}, index=Conversion_Previous_Yes.index)

# Rename the index column to 'previous'
conversion_previous_df.index.name = 'Calls Made Previous'

# Display the DataFrame
conversion_previous_df

,Conversion Percentage in Yes
Calls Made Previous,
0,8.832213
1,21.201491
2,46.419098
3,59.259259
4,54.285714
5,72.222222
6,60.000000
7,NaN


# 1. Descriptive Analysis 

In [21]:
# Assuming df is your DataFrame with numeric columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Convert DataFrame to numeric type and handle missing values
numeric_data = df.select_dtypes(include='number').dropna()

# Loop through each numeric column
for col in numeric_data.columns:
    print(f"\n_______  {col}  _______")
    print('-'*100)
    print()
    

    # Mean and Median
    mean_val = numeric_data[col].mean()
    median_val = numeric_data[col].median()
    print(f"Mean: {mean_val}")
    print(f"Median: {median_val}")
    print()

    # Skewness
    skewness = skew(numeric_data[col])
    print(f"Skewness: {skewness}")
    skewness_explanation = "Normally Distributed" if abs(skewness) <= 0.5 else "Not Normally Distributed"
    print(f"Skewness Explanation: {skewness_explanation}")
    print()

    # IQR (Interquartile Range)
    q1 = numeric_data[col].quantile(0.25)
    q3 = numeric_data[col].quantile(0.75)
    iqr_val = iqr(numeric_data[col])
    print(f"IQR: {iqr_val}")
    print()

    # Outlier Detection using IQR
    lower_bound = q1 - 1.5 * iqr_val
    upper_bound = q3 + 1.5 * iqr_val
    outliers = numeric_data[(numeric_data[col] < lower_bound) | (numeric_data[col] > upper_bound)]
    outlier_percentage = (len(outliers) / len(numeric_data)) * 100
    print(f"Outlier Percentage (using IQR): {outlier_percentage}%")
    print()

    # Z-Score Test for Outliers
    z_scores = zscore(numeric_data[col])
    z_outliers = numeric_data[abs(z_scores) > 3]
    z_outlier_percentage = (len(z_outliers) / len(numeric_data)) * 100
    print(f"Outlier Percentage (using Z-Score): {z_outlier_percentage}%")
    print()

    # Variance Inflation Factor (VIF) - Checking for multicollinearity
    vif_data = numeric_data.drop(col, axis=1)  # Exclude the current column from VIF calculation
    vif_values = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]
    vif_df = pd.DataFrame({'Variable': vif_data.columns, 'VIF': vif_values})
    vif_df['Multicollinearity'] = ['Yes' if vif > 10 else 'No' for vif in vif_values]
    print("\n--- Variance Inflation Factor (VIF) ---")
    print(vif_df)
    print()

    print(f"\n--- Univariate Analysis for {col} ---")
    print("Descriptive Statistics:")
    print(df[col].describe())
    print('**'*50)
    print()





_______  age  _______
----------------------------------------------------------------------------------------------------

Mean: 40.02406040594348
Median: 38.0

Skewness: 0.7846682380932289
Skewness Explanation: Not Normally Distributed

IQR: 15.0

Outlier Percentage (using IQR): 1.1386811692726035%

Outlier Percentage (using Z-Score): 0.8958920073807904%


--- Variance Inflation Factor (VIF) ---
         Variable           VIF Multicollinearity
0        duration      2.444702                No
1        campaign      1.921555                No
2           pdays     46.389563               Yes
3        previous      2.004558                No
4    emp.var.rate     28.941722               Yes
5  cons.price.idx  22842.836340               Yes
6   cons.conf.idx    119.543214               Yes
7       euribor3m    226.204059               Yes
8     nr.employed  26987.015363               Yes
9               y      1.664882                No


--- Univariate Analysis for age ---
Descriptiv

# Bivariate Test of T-Test For Finding Relationship

In [22]:
# Bivariate Analysis
print("\n--- Bivariate Analysis with T Test ---")
for col in numeric_data.columns[:-1]:
    t_stat, p_value = ttest_ind(numeric_data[numeric_data['y'] == 1][col], numeric_data[numeric_data['y'] == 0][col])
    print(f"T-Test p-value for {col}: {p_value}")
    
    # Interpretation
    if p_value < 0.05:
        print("Reject the Null Hypothesis. There is a significant difference.")
        print(" Negative correlation :--  doesn't seem strongly correlated with the target.")
        print()
        print('-' * 100)

    else:
        print("Fail to reject the Null Hypothesis. No significant difference observed.")
        print("Positive correlation.:--  doesn't seem strongly correlated with the target.")
        print()
        print('-' * 100)






--- Bivariate Analysis with T Test ---
T-Test p-value for age: 6.80213641846347e-10
Reject the Null Hypothesis. There is a significant difference.
 Negative correlation :--  doesn't seem strongly correlated with the target.

----------------------------------------------------------------------------------------------------
T-Test p-value for duration: 0.0
Reject the Null Hypothesis. There is a significant difference.
 Negative correlation :--  doesn't seem strongly correlated with the target.

----------------------------------------------------------------------------------------------------
T-Test p-value for campaign: 2.0077799906175707e-41
Reject the Null Hypothesis. There is a significant difference.
 Negative correlation :--  doesn't seem strongly correlated with the target.

----------------------------------------------------------------------------------------------------
T-Test p-value for pdays: 0.0
Reject the Null Hypothesis. There is a significant difference.
 Negative c

# Bivariate Test of Anova For Finding Relationship

In [23]:
# Bivariate Analysis with ANOVA
print("\n--- Bivariate Analysis with ANOVA ---")
for col in numeric_data.columns[:-1]:
    print(f"\n--- Bivariate Analysis for {col} and 'y' ---")

    # Perform ANOVA for numerical columns
    groups = [numeric_data[numeric_data['y'] == i][col] for i in numeric_data['y'].unique()]
    f_stat, p_value = f_oneway(*groups)
    print(f"ANOVA p-value for {col}: {p_value}")
    
    # Interpretation
    if p_value < 0.05:
        print("Reject the Null Hypothesis. There is a significant difference.")
        print("Negative correlation: Doesn't seem strongly correlated with the target.")
    else:
        print("Fail to reject the Null Hypothesis. No significant difference observed.")
        print("Positive correlation: Doesn't seem strongly correlated with the target.")
    print('-' * 100)


--- Bivariate Analysis with ANOVA ---

--- Bivariate Analysis for age and 'y' ---
ANOVA p-value for age: 6.802136418528356e-10
Reject the Null Hypothesis. There is a significant difference.
Negative correlation: Doesn't seem strongly correlated with the target.
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for duration and 'y' ---
ANOVA p-value for duration: 0.0
Reject the Null Hypothesis. There is a significant difference.
Negative correlation: Doesn't seem strongly correlated with the target.
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for campaign and 'y' ---
ANOVA p-value for campaign: 2.007779990638697e-41
Reject the Null Hypothesis. There is a significant difference.
Negative correlation: Doesn't seem strongly correlated with the target.
-------------------------------------------------------------------------------------

# Correlation Test For Finding Relationship

In [24]:
# Assuming numeric_data is your DataFrame with numeric columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Extract numeric columns
numeric_columns = numeric_data.iloc[:, :-1]

# Calculate correlation matrix
correlation_matrix = numeric_columns.corrwith(numeric_data['y'])

# Display correlation information in the specified format
print("\n--- Correlation with the Target Column 'y' ---")
for col, correlation_coefficient in correlation_matrix.items():
    #print(f"\n##### {col} ({correlation_coefficient:.6f}):")
    
    # Interpretation
    if abs(correlation_coefficient) <= 0.05:
        correlation_interpretation = "A very weak correlation."
    elif 0.05 < abs(correlation_coefficient) <= 0.2:
        correlation_interpretation = "A weak correlation."
    elif 0.2 < abs(correlation_coefficient) <= 0.5:
        correlation_interpretation = "A moderate correlation."
    else:
        correlation_interpretation = "A strong correlation."
    
    if correlation_coefficient > 0:
        correlation_interpretation += " The feature is positively correlated with the target."
    elif correlation_coefficient < 0:
        correlation_interpretation += " The feature is negatively correlated with the target."
    else:
        correlation_interpretation += " There is no linear correlation with the target."

    print()

    print(f" {col} ({correlation_coefficient:.6f}): \n {correlation_interpretation}")
    print()
    print('#' * 120)
    


--- Correlation with the Target Column 'y' ---

 age (0.030399): 
 A very weak correlation. The feature is positively correlated with the target.

########################################################################################################################

 duration (0.405274): 
 A moderate correlation. The feature is positively correlated with the target.

########################################################################################################################

 campaign (-0.066357): 
 A weak correlation. The feature is negatively correlated with the target.

########################################################################################################################

 pdays (-0.324914): 
 A moderate correlation. The feature is negatively correlated with the target.

########################################################################################################################

 previous (0.230181): 
 A moderate correlation. The feature 

In [25]:
from sklearn.feature_selection import mutual_info_classif

def analyze_mutual_information(data, target_column):
    """
    Analyze Mutual Information for each numeric column with the target column.

    Parameters:
    - data: DataFrame with numeric columns
    - target_column: Name of the target column

    Output:
    Prints the name of the column, its relation with the target column, the kind of relation,
    and whether the feature is informative or not.
    """

    print("\n--- Mutual Information Analysis ---")
    for col in data.columns[:-1]:
        print(f"\n--- Analysis for {col} ---")
        
        # Perform Mutual Information for numerical columns
        mutual_info = mutual_info_classif(data[[col]], data[target_column])[0]
        print(f"Mutual Information: {mutual_info}")

        # Interpretation
        if mutual_info > 0:
            print("Relation with Target: Positive correlation")
            print("Information Gain: The feature is informative for the target.")
        else:
            print("Relation with Target: No significant correlation")
            print("Information Gain: No significant information gain observed.")
        
        print('*' * 100)

# Example usage:
analyze_mutual_information(numeric_data, 'y')


--- Mutual Information Analysis ---

--- Analysis for age ---
Mutual Information: 0.014138679488393313
Relation with Target: Positive correlation
Information Gain: The feature is informative for the target.
****************************************************************************************************

--- Analysis for duration ---
Mutual Information: 0.0766872016110629
Relation with Target: Positive correlation
Information Gain: The feature is informative for the target.
****************************************************************************************************

--- Analysis for campaign ---
Mutual Information: 0.002274970662857756
Relation with Target: Positive correlation
Information Gain: The feature is informative for the target.
****************************************************************************************************

--- Analysis for pdays ---
Mutual Information: 0.03941065271526045
Relation with Target: Positive correlation
Information Gain: The featur

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming numeric_data is your DataFrame with numeric columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Split the data into features (X) and target (y)
X = numeric_data.iloc[:, :-1]
y = numeric_data['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model with increased max_iter
model = LogisticRegression(max_iter=1000)  # Adjust max_iter as needed
model.fit(X_train, y_train)

# Get coefficients
coefficients = model.coef_[0]

# Display coefficients and feature names
print("\n--- Logistic Regression Coefficients ---")
for col, coef in zip(X.columns, coefficients):
    print(f"{col}: {coef}")




--- Logistic Regression Coefficients ---
age: 0.0007417925074401134
duration: 0.004475231381796348
campaign: -0.06635424741518056
pdays: -0.0016538656467781734
previous: -0.09831649620321126
emp.var.rate: -0.22234666098452405
cons.price.idx: 0.3767101707418818
cons.conf.idx: 0.04397694776176596
euribor3m: -0.21868152108220795
nr.employed: -0.006786701227493292


# Logistic Regression For Finding Relationship

In [27]:
# Assuming numeric_data is your DataFrame with numeric columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Split the data into features (X) and target (y)
X = numeric_data.iloc[:, :-1]
y = numeric_data['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model with increased max_iter
model = LogisticRegression(max_iter=1000)  # Adjust max_iter as needed
model.fit(X_train, y_train)

# Get coefficients
coefficients = model.coef_[0]

# Display coefficients in a table format
print("| Variable         | Coefficient      | Interpretation                                                  ")
print("|------------------|------------------|-----------------------------------------------------------------")

for col, coef in zip(X.columns, coefficients):
    correlation_interpretation = ""
    
    # Interpretation
    if abs(coef) <= 0.05:
        correlation_interpretation = "Very weak correlation; not strongly correlated"
    elif 0.05 < abs(coef) <= 0.2:
        correlation_interpretation = "Weak correlation"
    elif 0.2 < abs(coef) <= 0.5:
        correlation_interpretation = "Moderate correlation"
    else:
        correlation_interpretation = "Strong correlation"
    
    if coef > 0:
        correlation_interpretation += "; positively correlated."
    elif coef < 0:
        correlation_interpretation += "; negatively correlated."
    else:
        correlation_interpretation += "; no linear correlation."

    print(f"| {col:<16} | {coef:<12.6f}     | {correlation_interpretation:<56}")

| Variable         | Coefficient      | Interpretation                                                  
|------------------|------------------|-----------------------------------------------------------------
| age              | 0.000742         | Very weak correlation; not strongly correlated; positively correlated.
| duration         | 0.004475         | Very weak correlation; not strongly correlated; positively correlated.
| campaign         | -0.066354        | Weak correlation; negatively correlated.                
| pdays            | -0.001654        | Very weak correlation; not strongly correlated; negatively correlated.
| previous         | -0.098316        | Weak correlation; negatively correlated.                
| emp.var.rate     | -0.222347        | Moderate correlation; negatively correlated.            
| cons.price.idx   | 0.376710         | Moderate correlation; positively correlated.            
| cons.conf.idx    | 0.043977         | Very weak correlation; not st

# Random Forest Classifier For Finding Relationship

In [28]:
# Assuming numeric_data is your DataFrame with numeric columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Split the data into features (X) and target (y)
X = numeric_data.iloc[:, :-1]
y = numeric_data['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame to store feature importances
feature_importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': importances})

# Display feature importances in a table format
print("| Variable         | Importance      | Interpretation                                                 |")
print("|------------------|------------------|-----------------------------------------------------------------|")

for index, row in feature_importance_df.iterrows():
    importance_interpretation = ""
    
    # Interpretation
    if row['Importance'] <= 0.05:
        importance_interpretation = "Very weak importance; not strongly influential"
    elif 0.05 < row['Importance'] <= 0.2:
        importance_interpretation = "Weak importance"
    elif 0.2 < row['Importance'] <= 0.5:
        importance_interpretation = "Moderate importance"
    else:
        importance_interpretation = "Strong importance"
    
    #importance_interpretation += f"; {row['Importance']:.6f} proportion of total importance."

    print(f"| {row['Variable']:<16} | {row['Importance']:<12.6f}     | {importance_interpretation:<60} |")

| Variable         | Importance      | Interpretation                                                 |
|------------------|------------------|-----------------------------------------------------------------|
| age              | 0.152906         | Weak importance                                              |
| duration         | 0.411514         | Moderate importance                                          |
| campaign         | 0.057237         | Weak importance                                              |
| pdays            | 0.047141         | Very weak importance; not strongly influential               |
| previous         | 0.020016         | Very weak importance; not strongly influential               |
| emp.var.rate     | 0.019464         | Very weak importance; not strongly influential               |
| cons.price.idx   | 0.025992         | Very weak importance; not strongly influential               |
| cons.conf.idx    | 0.028480         | Very weak importance; not str

# Statical Analysis on Categorical Values

In [29]:
categorical_data['y']=numeric_data['y']

# categorical_data['y']=numeric_data['y']
# categorical_data

In [30]:
categorical_data.head(5)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,0
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,0
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,0
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,0
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,0


In [31]:
categorical_data.tail(5)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
41183,retired,married,professional.course,no,yes,no,cellular,nov,fri,nonexistent,1
41184,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,0
41185,retired,married,university.degree,no,yes,no,cellular,nov,fri,nonexistent,0
41186,technician,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,1
41187,retired,married,professional.course,no,yes,no,cellular,nov,fri,failure,0


In [32]:
categorical_data.shape

(41188, 11)

In [33]:
categorical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job          41188 non-null  object
 1   marital      41188 non-null  object
 2   education    41188 non-null  object
 3   default      41188 non-null  object
 4   housing      41188 non-null  object
 5   loan         41188 non-null  object
 6   contact      41188 non-null  object
 7   month        41188 non-null  object
 8   day_of_week  41188 non-null  object
 9   poutcome     41188 non-null  object
 10  y            41188 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 3.5+ MB


## Univariate 

In [34]:
categorical_data.head(5)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,0
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,0
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,0
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,0
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,0


In [35]:
categorical_data.tail(5)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
41183,retired,married,professional.course,no,yes,no,cellular,nov,fri,nonexistent,1
41184,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,0
41185,retired,married,university.degree,no,yes,no,cellular,nov,fri,nonexistent,0
41186,technician,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,1
41187,retired,married,professional.course,no,yes,no,cellular,nov,fri,failure,0


# Count Of Each Class with Target Yes And No

In [36]:
# Assuming df is your DataFrame with categorical columns and the last column is 'y'
# Replace 'y' with your actual target column name

# Get the categorical columns
categorical_data = df.select_dtypes(include='object').dropna()

# Loop through each categorical column
for col in categorical_data.columns:
    print(f"\n{'='*60}\n{' '*20}Frequency Table for {col.capitalize()}\n{'='*60}")

    # Frequency count
    frequency_table = df[col].value_counts()

    # Print table header
    print(f"| {'Class': <30} | {'Total Count': <15} | {'Count with Yes': <15} | {'Count with No': <15} |")
    print(f"|{'-'*31}|{'-'*16}|{'-'*16}|{'-'*16}|")

    # Print rows
    for value in frequency_table.index:
        class_counts = df[df[col] == value]['y'].value_counts()

        count_total = frequency_table[value]
        count_yes = class_counts.get(1, 0)
        count_no = class_counts.get(0, 0)

        print(f"| {value: <30} | {count_total: <15} | {count_yes: <15} | {count_no: <15} |")

    print('\n')


                    Frequency Table for Job
| Class                          | Total Count     | Count with Yes  | Count with No   |
|-------------------------------|----------------|----------------|----------------|
| admin.                         | 10422           | 1352            | 9070            |
| blue-collar                    | 9254            | 638             | 8616            |
| technician                     | 6743            | 730             | 6013            |
| services                       | 3969            | 323             | 3646            |
| management                     | 2924            | 328             | 2596            |
| retired                        | 1720            | 434             | 1286            |
| entrepreneur                   | 1456            | 124             | 1332            |
| self-employed                  | 1421            | 149             | 1272            |
| housemaid                      | 1060            | 106             

# Class Distibution

In [37]:
# Loop through each categorical column
for col in categorical_data.columns:
    print(f"\n{'='*80}\n{' '*20}Class Distribution for {col.capitalize()}\n{'='*80}")
    
    # Frequency count
    frequency_table = df[col].value_counts()

    # Percentage of each class with the target column
    print(f"| {'Class': <20} | {'Perc. with YES': <20}     | {'Perc. with NO': <20} |")
    print(f"|{'-'*21} |{'-'*21}     |{'-'*21}|")
    for value in frequency_table.index:
        class_counts = df[df[col] == value]['y'].value_counts()
        total_counts = class_counts.sum()

        count_yes = class_counts.get(1, 0)
        count_no = class_counts.get(0, 0)

        # Percentage difference
        percentage_yes = count_yes / total_counts * 100
        percentage_no = count_no / total_counts * 100

        print(f"| {value: <20} | {percentage_yes:.6f}%                | {percentage_no:.6f}% |")

    print('\n')

# Add a separator between tables
print('='*80)


                    Class Distribution for Job
| Class                | Perc. with YES           | Perc. with NO        |
|--------------------- |---------------------     |---------------------|
| admin.               | 12.972558%                | 87.027442% |
| blue-collar          | 6.894316%                | 93.105684% |
| technician           | 10.826042%                | 89.173958% |
| services             | 8.138070%                | 91.861930% |
| management           | 11.217510%                | 88.782490% |
| retired              | 25.232558%                | 74.767442% |
| entrepreneur         | 8.516484%                | 91.483516% |
| self-employed        | 10.485574%                | 89.514426% |
| housemaid            | 10.000000%                | 90.000000% |
| unemployed           | 14.201183%                | 85.798817% |
| student              | 31.428571%                | 68.571429% |
| unknown              | 11.212121%                | 88.787879% |



           

# Minimum, Maximum, Mean With Yes and No

In [38]:
for col in categorical_data.columns:
    print("\nMean, Min, Max Regarding Yes and No:")
    print()
    print(f"|{col: <15}|{'Class': <15}|{'Mean with YES': <15}|{'Mean with NO': <15}|"
          f"{'Min with YES': <15}|{'Min with NO': <15}|{'Max with YES': <15}|{'Max with NO': <15}|")
    print('-'*100)

    # Frequency count
    frequency_table = df[col].value_counts()

    # Statistics for both Yes and No
    for value in frequency_table.index:
        class_counts = df[df[col] == value]['y'].value_counts()
        total_counts = class_counts.sum()

        count_yes = class_counts.get(1, 0)
        count_no = class_counts.get(0, 0)

        # Mean, Minimum, and Maximum for each class
        mean_value_yes = count_yes / total_counts
        mean_value_no = count_no / total_counts
        min_value_yes = class_counts.min()
        min_value_no = class_counts.min()
        max_value_yes = class_counts.max()
        max_value_no = class_counts.max()

        print(f"|{value: <15}|{mean_value_yes * 100: <15.6f}%|{mean_value_no * 100: <15.6f}%|"
              f"{min_value_yes: <15}|{min_value_no: <15}|{max_value_yes: <15}|{max_value_no: <15}|")

    # Add a separator between classes
    print('-' * 105)

# Add a separator between tables# Loop through each categorical column
for col in categorical_data.columns:
    print("\nMean, Min, Max Regarding Yes and No:")
    print(f"|{col: <15}|{'Class': <15}|{'Mean with T1': <15}|{'Mean with T0': <15}|"
          f"{'Min with T1': <15}|{'Min with T0': <15}|{'Max with T1': <15}|{'Max with T0': <15}|")

    # Frequency count
    frequency_table = df[col].value_counts()

    # Statistics for both Yes and No
    for value in frequency_table.index:
        class_counts = df[df[col] == value]['y'].value_counts()
        total_counts = class_counts.sum()

        count_yes = class_counts.get(1, 0)
        count_no = class_counts.get(0, 0)

        # Mean, Minimum, and Maximum for each class
        mean_value_yes = count_yes / total_counts
        mean_value_no = count_no / total_counts
        min_value_yes = class_counts.min()
        min_value_no = class_counts.min()
        max_value_yes = class_counts.max()
        max_value_no = class_counts.max()

        print(f"|  {value: <15}    |{mean_value_yes * 100: <15.6f}%    |{mean_value_no * 100: <15.6f}%    |"
              f"{min_value_yes: <15}     |{max_value_yes: <15}  ")

    # Add a separator between classes
    print('-' * 105)

# Add a separator between tables
print('\n' + '*' * 100 + '\n')
print('\n' + '*' * 100 + '\n')


Mean, Min, Max Regarding Yes and No:

|job            |Class          |Mean with YES  |Mean with NO   |Min with YES   |Min with NO    |Max with YES   |Max with NO    |
----------------------------------------------------------------------------------------------------
|admin.         |12.972558      %|87.027442      %|1352           |1352           |9070           |9070           |
|blue-collar    |6.894316       %|93.105684      %|638            |638            |8616           |8616           |
|technician     |10.826042      %|89.173958      %|730            |730            |6013           |6013           |
|services       |8.138070       %|91.861930      %|323            |323            |3646           |3646           |
|management     |11.217510      %|88.782490      %|328            |328            |2596           |2596           |
|retired        |25.232558      %|74.767442      %|434            |434            |1286           |1286           |
|entrepreneur   |8.516484       %|

# Bivariate Analysis with Chi-square Test


In [39]:
categorical_data['y']=numeric_data['y']
categorical_data

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,0
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,0
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,0
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,0
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,0
...,...,...,...,...,...,...,...,...,...,...,...
41183,retired,married,professional.course,no,yes,no,cellular,nov,fri,nonexistent,1
41184,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,0
41185,retired,married,university.degree,no,yes,no,cellular,nov,fri,nonexistent,0
41186,technician,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,1


In [40]:
print("\n--- Bivariate Analysis with Chi-square Test ---")
for col in categorical_data[:-1]:  # Exclude the target column 'y'
    print(f"\n--- Bivariate Analysis for {col} and 'y' ---")

    # Create a contingency table
    contingency_table = pd.crosstab(categorical_data[col], categorical_data['y'])
    
    # Perform Chi-square test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    # Interpretation
    if p_value < 0.05:
        print("Reject the Null Hypothesis. There is a significant association with the target (y).")
    else:
        print("Fail to reject the Null Hypothesis. No significant association observed.")
    
    print(f"Chi-square value for {col}: {chi2}")
    print(f"P-value: {p_value}")
    print('-' * 100)


--- Bivariate Analysis with Chi-square Test ---

--- Bivariate Analysis for job and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
Chi-square value for job: 961.2424403289555
P-value: 4.189763287563623e-199
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for marital and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
Chi-square value for marital: 122.65515182252989
P-value: 2.068014648442211e-26
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for education and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
Chi-square value for education: 193.10590454149565
P-value: 3.3051890144025054e-38
----------------------------------------------------------------------------------------------------

--- Bivariate 

# Bivariate Analysis with G-test


In [41]:
print("\n--- Bivariate Analysis with G-test (Likelihood Ratio Test) ---")
for col in categorical_data.columns[:-1]:  # Exclude the target column 'y'
    print(f"\n--- Bivariate Analysis for {col} and 'y' ---")

    # Create a contingency table
    contingency_table = pd.crosstab(categorical_data[col], categorical_data['y'])
    
    # Perform G-test (Likelihood Ratio Test)
    g_stat, p_value, _, _ = chi2_contingency(contingency_table, lambda_='log-likelihood')
    
    # Interpretation
    if p_value < 0.05:
        print("Reject the Null Hypothesis. There is a significant association with the target (y).")
    else:
        print("Fail to reject the Null Hypothesis. No significant association observed.")
    
    print(f"G-test (Likelihood Ratio Test) statistic for {col}: {g_stat}")
    print(f"P-value: {p_value}")
    print('-' * 100)


--- Bivariate Analysis with G-test (Likelihood Ratio Test) ---

--- Bivariate Analysis for job and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
G-test (Likelihood Ratio Test) statistic for job: 812.1222750903776
P-value: 4.726296851106596e-167
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for marital and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
G-test (Likelihood Ratio Test) statistic for marital: 118.11493261710558
P-value: 1.9651333130891509e-25
----------------------------------------------------------------------------------------------------

--- Bivariate Analysis for education and 'y' ---
Reject the Null Hypothesis. There is a significant association with the target (y).
G-test (Likelihood Ratio Test) statistic for education: 196.85313003757153
P-value: 5.322901865315799e-39
-----------------------------